# Deep learning techniques to identify the composer of a music piece

**Objective**  
The primary objective of this project is to develop a deep learning model that can predict the composer of a given musical score accurately. The project aims to accomplish this objective by using two deep learning techniques: Long Short-Term Memory (LSTM) and Convolutional Neural Network (CNN).

**Dataset**  
The project will use a dataset consisting of musical scores from various composers. The dataset contain MIDI files and sheet music of compositions from well-known classical composers like Bach, Beethoven, Chopin, Mozart, Schubert, etc. The dataset should be labeled with the name of the composer for each score.

## Data Collection

### Install required libraries

In [1]:
# Import all dependent libraries
import os
import csv
import zipfile
import shutil

from tqdm import tqdm

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import math
import keras.optimizers
import tensorflow as tf


# Music related libraries
import music21
from music21 import converter, instrument, note, chord, tempo

# Machine Learning Libraries
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.model_selection import TimeSeriesSplit

# Deep Learning Libraries
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM, Activation
from sklearn.preprocessing import MinMaxScaler
from keras.utils import pad_sequences # not used , should we remove it?
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Suppress warnings for cleaner output
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [2]:
from google.colab import drive

def is_runnong_on_colab():
  return 'google.colab' in str(get_ipython())

if is_runnong_on_colab():
  print('Running on Colab')
  from google.colab import drive
  drive.mount('/content/drive' , force_remount=True)
  root_path = '/content/drive/MyDrive/AAI-511-IN2 Neural Networks and Deep Learning/Project'
else:
  print('Not running on Colab')

Running on Colab
Mounted at /content/drive


In [3]:
# Root path
root_path = '/content/drive/MyDrive/AAI-511-IN2 Neural Networks and Deep Learning/Project'

# File path in Google Drive
file_path = os.path.join(root_path, 'Composer_Dataset.zip')

# Extract file path
extract_path = os.path.join(root_path, 'Composer_Dataset')

# Dataset folders
dataset_path = os.path.join(extract_path, 'Composer_Dataset/NN_midi_files_extended')

# CSV index file
csv_file = os.path.join(dataset_path, 'composer_dataset_index.csv')

In [ ]:
# Check if folder already exists
if not os.path.exists(extract_path):
    print("Extracting dataset...")
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
else:
    print("Dataset already extracted.")

Dataset already extracted.


In [ ]:
def delete_hidden_folders(root_folder):
    for root, dirs, _ in os.walk(root_folder):
        for d in dirs:
            if d.startswith('.'):
                dir_path = os.path.join(root, d)
                print(f"Deleting hidden folder: {dir_path}")
                shutil.rmtree(dir_path)
            elif d == '__MACOSX':
                dir_path = os.path.join(root, d)
                print(f"Deleting __MACOSX folder: {dir_path}")
                shutil.rmtree(dir_path)

# Run this on the outer Composer_Dataset
delete_hidden_folders(extract_path)

## Data Pre-processing

In [ ]:
# Function to create index files
def create_midi_file_index_csv(root_dir, output_csv):
    rows = []

    for split in ['train', 'test', 'dev']:
        split_path = os.path.join(root_dir, split)
        if not os.path.isdir(split_path):
            continue

        for composer in os.listdir(split_path):
            composer_path = os.path.join(split_path, composer)
            if not os.path.isdir(composer_path) or composer.startswith('.'):
                continue

            for filename in os.listdir(composer_path):
                if not filename.endswith('.mid') or filename.startswith('.'):
                    continue

                filepath = os.path.join(split, composer, filename)
                rows.append({
                    'split': split,
                    'composer': composer,
                    'filename': filename,
                    'filepath': filepath
                })

    # Write to CSV
    with open(output_csv, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['split', 'composer', 'filename', 'filepath'])
        writer.writeheader()
        writer.writerows(rows)

    print(f"CSV created: {output_csv} with {len(rows)} entries.")

# Create a csv file
create_midi_file_index_csv(
    root_dir=dataset_path,
    output_csv=csv_file
)

CSV created: /content/drive/MyDrive/AAI-511-IN2 Neural Networks and Deep Learning/Project/Composer_Dataset/Composer_Dataset/NN_midi_files_extended/composer_dataset_index.csv with 439 entries.


## Feature Extraction

In [ ]:
# Function to extract features
def extract_features_from_midi(filepath):
    try:
        midi = converter.parse(filepath)

        # Get tempo (default to 120 if not found)
        bpm = 120
        tempo_indications = midi.flat.getElementsByClass(tempo.MetronomeMark)
        if tempo_indications:
            bpm = tempo_indications[0].number

        notes = []
        durations = []

        for element in midi.recurse():
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
                durations.append(element.duration.quarterLength)
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                durations.append(element.duration.quarterLength)

        return {
            'num_events': len(notes),
            'unique_events': len(set(notes)),
            'avg_duration': sum(durations) / len(durations) if durations else 0,
            'tempo': bpm,
            'note_sequence': ' '.join(notes[:100])  # first 100 events for preview
        }
    except Exception as e:
        return {
            'num_events': 0,
            'unique_events': 0,
            'avg_duration': 0,
            'tempo': 0,
            'note_sequence': '',
            'error': str(e)
        }

In [ ]:
# Load your previously generated CSV
df = pd.read_csv(csv_file)

# Collect features
features = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    full_path = os.path.join(dataset_path, row['filepath'])
    feature = extract_features_from_midi(full_path)
    feature.update({
        'split': row['split'],
        'composer': row['composer'],
        'filename': row['filename'],
        'filepath': row['filepath']
    })
    features.append(feature)

# Save to a new DataFrame
features_df = pd.DataFrame(features)

# Write it to the csv file
features_df.to_csv(csv_file, index=False)

# Print the extracted features
features_df.head()

  0%|          | 0/439 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/music21/stream/base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)
  8%|▊         | 33/439 [05:58<1:51:45, 16.52s/it]/usr/local/lib/python3.11/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'La Fl\xfbte Enchant\xe9e (Die Zauberfl\xf6te) - W.A. Mozart'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'Acte II, Aria N\xb014'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.mi

,num_events,unique_events,avg_duration,tempo,note_sequence,split,composer,filename,filepath
0,4027,140,0.507802,114.0,D5 E-5 F5 F5 F5 G5 F5 E5 F5 G5 F5 E-5 D5 B-5 A...,train,mozart,mozart048.mid,train/mozart/mozart048.mid
1,3903,145,0.540930,112.0,D6 D6 D6 D6 B5 G5 D5 D6 B5 C6 A5 F#5 D5 C6 A5 ...,train,mozart,mozart028.mid,train/mozart/mozart028.mid
2,18911,171,0.665212,116.0,C6 G5 E5 C5 E5 G5 C6 G5 E5 C5 E5 G5 C6 C5 B4 C...,train,mozart,mozart000.mid,train/mozart/mozart000.mid
3,4846,131,0.443459,41.0,3.7.10 B-4 0.2 B-4 B-5 D5 E-5 10.2.5 B-4 0.2 B...,train,mozart,mozart001.mid,train/mozart/mozart001.mid
4,2178,155,0.607515,76.0,F5 C5 A5 F5 C6 B-5 A5 G5 F5 F#5 G5 C5 E5 G5 B-...,train,mozart,mozart015.mid,train/mozart/mozart015.mid


## Model Building

## Model Training

## Model Evaluation

## Model Optimization